In [1]:
#-*- coding: UTF-8 -*-

# 加载数据
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5],[3,1,5,2],[1,2,3]]

# 将所有元素转换为frozenset型字典，存放到列表中
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:        #去重得到单元素项
                C1.append([item])
    C1.sort()                           #排序

    return list(map(frozenset, C1))  # 使用frozenset是为了后面可以将这些值作为字典的键 frozenset一种不可变的集合，set可变集合


# 过滤掉支持度不符合的集合 即剪枝
def scanD(D, Ck, minSupport):
    ssCnt = {}                      #建立字典存储各项及数目
    for tid in D:
        for can in Ck:
            if can.issubset(tid):  # 判断can是否是tid的《子集》 （这里使用子集的方式来判断两者的关系）
                ssCnt[can] = ssCnt.get(can,0)+1   #读取字典 存在can得到数值 不存在得到零 在取到的值加1 .get(要取的值，没有返回的值)
    retList = []  # 记录支持度大于给定值的数据
    supportData = {}  # 每个数据值的支持度
    for key in ssCnt:
        support = ssCnt[key] / len(D)     #计算支持度
        if support >= minSupport:           #和给定支持度比较
            retList.append(key);supportData[key] = support  #将符合支持度条件的数据存储
    return retList, supportData  # 排除不符合支持度元素后的元素 每个元素支持度# 返回频繁项集列表retList 所有元素的支持度字典


# 生成所有可以组合的集合
# 频繁项集列表Lk
# 连接后项集元素个数k  [frozenset({2, 3}), frozenset({3, 5})] -> [frozenset({2, 3, 5})]
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):  # 两层循环比较Lk中的每个元素与其它元素
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k - 2]  # 将集合转为list后 取0-k-2
            L2 = list(Lk[j])[:k - 2]
            L1.sort();
            L2.sort()
            if L1 == L2:# 该函数每次比较两个list的前k-2个元素，如果相同则求并集得到k个元素的集合
                retList.append(Lk[i] | Lk[j])  # 求并集python求并集符号|
    return retList  # 返回连接后的项集


# 调用上述函数
# 返回 所有满足大于阈值的组合 集合支持度列表
def apriori(dataSet, minSupport):
    D = list(map(set, dataSet))  # 转换列表记录为字典  [{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
    C1 = createC1(dataSet)  # 将每个元素转会为frozenset字典    [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    L1, supportData = scanD(D, C1, minSupport)  # 过滤数据
    L = [L1]   #频繁项集的集合
    k = 2
    while (len(L[k-2]) > 0):  # 若仍有满足支持度的集合则继续做关联分析
        Ck = aprioriGen(L[k-2], k)  # Ck候选频繁项集
        Lk, supK = scanD(D, Ck, minSupport)  # Lk频繁项集 supk支持度
        L.append(Lk)  # 更新L集合
        supportData.update(supK)   # 更新字典（把新出现的集合:支持度加入到supportData中）
        k=k+1  # 每次新组合的元素都只增加了一个，所以k也+1（k表示元素个数）
    support = {}
    for i in L[k - 3]:     #遍历符合支持度元素最多的项
        support[i] = supportData.get(i)   #从字典中取出项及其支持度
    return support   #返回项及其支持度

if __name__ == '__main__':
    while int(input('请输入序号：1 测试 0 退出\n')):
        print('输入支持度(0->1)')
        support = float(input())
        print(support)        #定义float型输入支持度
        supp= apriori(loadDataSet(),support)
        print(supp)                  #打印结果



输入支持度(0->1)
0.777
{frozenset({3}): 0.8333333333333334, frozenset({2}): 0.8333333333333334}
输入支持度(0->1)
0.666
{frozenset({1, 3}): 0.6666666666666666, frozenset({2, 3}): 0.6666666666666666, frozenset({2, 5}): 0.6666666666666666}
输入支持度(0->1)
0.555
{frozenset({1, 3}): 0.6666666666666666, frozenset({2, 3}): 0.6666666666666666, frozenset({2, 5}): 0.6666666666666666}
